In [6]:
# -*- coding: utf-8 -*-
import scrapy
import re


class GetdataSpider(scrapy.Spider):
    usedUrl = set()
    url = 'http://www.tianqihoubao.com'
    name = 'getdata'
    allowed_domains = ['www.tianqihoubao.com']
    start_urls = ['http://www.tianqihoubao.com/lishi/beijing/month/201812.html']

    def parse(self, response):
        print("hello ")
        trs = response.xpath('//tr')
        isFirst = True
        invisibleChar = re.compile(r'[\s]+')
        nonDigit = re.compile(r'[^\d]+')
        for tr in trs:
            if isFirst:
                isFirst = False
                continue
            tds = tr.xpath('./td')
            date = re.sub(invisibleChar, "",tds[0].xpath('./a/text()').extract()[0])
            date = re.sub(nonDigit, '-', date)[0:-1]
            yield {
                "date":date,
                "weather":re.sub(invisibleChar, "",tds[1].xpath('./text()').extract()[0]),
                "temperature":re.sub(invisibleChar, "",tds[2].xpath('./text()').extract()[0]),
                "wind":re.sub(invisibleChar, "",tds[3].xpath('./text()').extract()[0])
            }
        nextUrls = response.xpath('//div[@class="months"]/a/@href')
        self.usedUrl.add(response.url)
        for nextUrl in nextUrls: 
            url = self.url+nextUrl.extract()
            if url in self.usedUrl:
                continue
            print("*********" + url)
            yield scrapy.Request(url, callback=self.parse)
            break

spider = GetdataSpider()
url = 'http://www.tianqihoubao.com/lishi/beijing/month/201811.html'
response=scrapy.Request(url, callback=spider.parse)
    

In [9]:
type(response)
response.

scrapy.http.request.Request

AttributeError: 'Request' object has no attribute 'xpath'